1. date_opt에서 서버로 날짜(day), 시간(time), 자치구(gu) 정보를 넘김
2. 서버는 수집한 날씨와 미세먼지를 예측 함수에 넘겨줌

3. 일기예보, 시간, 자치구를 넘겨받아 유동인구 예측
    - RAIN	HUMN	SNOW	SKY	ONDO	WINDD	WINDS	PM10	PM25 T
    - W(가중치)는 저장된 정보를 활용
    
4. 예측 후 유동인구 크기 순으로 내림차순 정렬

5. subway_opt에 result를 넘겨줌



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import joblib


In [2]:
from sklearn.preprocessing import MinMaxScaler , PolynomialFeatures
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
# 한글 문제
# matplotlit의 기본 폰트에서 한글 지원되지 않기 때문에
# matplotlib의 폰트 변경 필요
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

In [52]:
# 모델 로드
lr = joblib.load('weater_lr.pkl')

In [6]:
# 자치구별 지하철역과 가중치 데이터 로드
station_W = pd.read_csv('./data/station_W.csv')

In [65]:
# 넘어온 정보를 가중치와 병합, 길이를 맞춤
def pre_predict(wea,gu):
    weather = pd.DataFrame()

    W = station_W[station_W['GU'] == gu].set_index('STATION_NAME',drop=False)[['W']]
    wea = pd.DataFrame(wea*13).loc[:len(wea)-1].T
    W = np.log1p(W)
    # gu에서 받아온 역의 갯수만큼 날씨변수 생성
    for i in range(len(W)):
        weather = pd.concat([weather,wea],axis=0)
        # gu에서 받아온 역의 갯수만큼 날씨변수 생성
    weather.index = W.index
    values = pd.concat([np.log1p(weather),W], axis=1)
    values.index = W.index# 날씨와 가중치 컬럼 합병
    return values

In [73]:
def predict(values):
    result = lr.predict(values)
    result = pd.DataFrame(np.expm1(result))
    result.index = values.index
    result.columns = ['result']
#     result.index = values.index
#     result.sort_values(ascending=False, inplace=True)
    
    return result

In [74]:
# RAIN	HUMN	SNOW	SKY	ONDO	WINDD	WINDS	PM10	PM25	
# 22년 11월 6일 17시경의 예보
wea = [0,45,0,0,11,13,0,40, 18,1]

In [75]:

values = pre_predict(wea,'강남구')

In [76]:
result = predict(values)

/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/parkjunsoo/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [78]:
result.sort_values(by='result', ascending=False)

,result
STATION_NAME,
강남,9500.888936
삼성,5082.115324
선릉,5048.740287
역삼,4781.001542
신사,3613.093778
압구정,3361.990089
수서,1988.039803
학동,1969.410345
청담,1937.474486


In [79]:
station_W

,Unnamed: 0,STATION_NAME,W,GU
0,0,가락시장,2559.875000,송파구
1,1,가산디지털단지,3064.296975,금천구
2,2,강남,7786.252106,강남구
3,3,강남구청,1422.483296,강남구
4,4,강동,1514.337030,강동구
...,...,...,...,...
218,218,홍제,2077.179253,서대문구
219,219,화곡,2393.328678,강서구
220,220,화랑대,1327.415707,노원구
221,221,회현,2335.083462,중구
